In [224]:
import pandas as pd
import numpy as np
import nltk, datetime, re, warnings
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz, process
from IPython.display import HTML

In [125]:
#Run notebook in conda env lexnlp
import lexnlp.extract.en.definitions
import lexnlp.extract.en.amounts
import lexnlp.extract.en.regulations

/Users/katherinewood/anaconda3/envs/lexnlp/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [25]:
def test_all_options(string1, string2):
    ratio = fuzz.ratio(string1, string2)
    partial_ratio = fuzz.partial_ratio(string1, string2)
    token_sort = fuzz.token_sort_ratio(string1, string2)
    token_set = fuzz.token_set_ratio(string1, string2)
    print(string1 + ', ' + string2 + '\n' +
        'ratio: ' + str(ratio) + '\n'
        'partial_ratio: ' + str(ratio) + '\n'
        'token_sort: ' + str(ratio) + '\n'
        'token_set: ' + str(ratio) + '\n')

In [212]:
'SKU33' in 'SKU33'

True

In [361]:
pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 0)
pd.set_option('expand_frame_repr', False)

In [146]:
recalls = pd.read_csv('recalls.csv', 
                      dtype={'RecallID':'Int64', 'RecallNumber': str, 'RecallDate': str,
                             'Description': str, 'URL': str, 'Title': str, 'ConsumerContact': str,
                             'LastPublishDate': str, 'Images': 'object', 'SoldAtLabel': str,
                             'Distributors_CompanyID': 'Int64', 'DistributorsSName': str, 
                             'Hazards_HazardType': str, 'Hazards_HazardTypeID': str, 'Hazards_Name': str,
                             'Importers_CompanyID': 'Int64', 'Importers_Name': str, 'Inconjunctions_URL': str,
                             'Injuries_Name': str, 'ManufacturerCountries_Country': str, 
                             'Manufacturers_CompanyID': 'Int64', 'Manufacturers_Name': str, 'ProductUPCs_UPC': str,
                             'Products_CategoryID': 'Int64', 'Products_Description': str, 'Products_Model': str,
                             'Products_Name': str, 'Products_NumberOfUnits': str, 'Products_Type': str,
                             'Remedies_Name': str, 'RemedyOptions_Option': str, 'Retailers_CompanyID': str,
                             'Retailers_Name': str})

In [150]:
#Munge and infill whatever information we can extract from the recall descriptions

##UPCs loaded as strings; clean for whitespace & non-numeric chars
recalls['ProductUPCs_UPC'] = recalls['ProductUPCs_UPC'].str.replace(' |-|\.', '')
recalls = recalls.rename(columns={'ProductUPCs_UPC': 'UPC'})
#Extract unit numbers from string phrases (e.g. "About 35")
num_units = recalls['Products_NumberOfUnits'].str.replace(',', '')
num_units = num_units.str.extract(r'(\d+)', expand=False).astype('float')
recalls['Products_NumberOfUnits'] = num_units
#Extract total number of complaints from the string column
#Parse dates from strings
recalls['RecallDate'] = pd.to_datetime(recalls['RecallDate'])
recalls['LastPublishDate'] = pd.to_datetime(recalls['LastPublishDate'])
#Break the standardized titles into helpful fields; standardized
#titles take the form "[Company] recalls [product] due to [hazard]"
titles = recalls['Title'].str.split('[Re]?[A]?[a]?[nnounce]?[s]?Recall[s]?[ed]?|Due to', expand=True)
titles = titles.rename(columns={0: 'CompanyShortname', 1: 'ProductsShortname', 2: 'HazardAlt'})
recalls = pd.concat([recalls, titles], axis=1)

In [68]:
#Vectorize over a series of strings
def extract_probable_specifiers(text):
    if pd.isnull(text):
        text=''
    pattern = r"(([0-9A-Z])+[a-z]*([\\-]?[\\.*]?[0-9A-Z]*)*){2,}"
    matches = re.finditer(pattern, text)
    unique_matches = set([match.group() for matchNum, match in enumerate(matches)])
    return list(unique_matches)

In [405]:
# TODO combine vornado and bosch for a test set
reports = pd.concat([v_reports, b_reports], axis=0, sort=False)
recalls = pd.concat([v_recalls, b_recalls], axis=0, sort=False)

#Funnel match
#Preparation: extract possible brands from the fields likely to contain them, for both the reports and the recalls.
#Brands may not be referred to by a consistent name across the two datasets.
brand_from_comments = [report[1] if not isinstance(report, float) else '' for report in reports['Company Comments'].str.split('\\) |:')]
reports['candidate_brand'] = list(zip(reports['Brand'], 
                                           reports['Manufacturer / Importer / Private Labeler Name'],
                                           brand_from_comments))
reports['candidate_brand'] = reports['candidate_brand'].apply(lambda x: list(set(x)))
recalls['candidate_brand'] = list(zip(recalls['CompanyShortname'], recalls['Manufacturers_Name'],
                                     recalls['Distributors_Name'], recalls['Importers_Name']))
recalls['candidate_brand'] = recalls['candidate_brand'].apply(lambda x: list(set(x)))

#Preparation: extract alphanumeric strings that are likely to be model numbers, serial numbers, or UPCs.
#Recall notices are very unlikely to have the dedicated fields populated, but tend to mention them in the
#text of the recall announcement.
#Reports tend to have this information in the dedicated fields, but as a precaution we also try to pull it
#from the unstructured text.

reports['specifiers'] = [extract_probable_specifiers(report) for report in 
                         [reports['Product Description'] + ' '+ reports['Incident Description']][0]]

recalls['specifiers'] = [extract_probable_specifiers(recall) for recall in recalls['Description']]

#Phase 1
#For each complaint:
#for each candidate brand: 'CompanyShortname', 'Manufacturers_Name', 'Distributors_Name', 'Importers_Name'
# - Fuzzy match to the possible brands list from each recall notice
# - If any matches score > threshold, save recall as candidate (add a column that contains a list of probable recall IDs)
# - If no matches score > threshold, label complaint as "no recall"
#Phase 2
#For all complaints that have candidate recall numbers:
#For each candidate recall:
# - Fuzzy match to possible products using product type from report & product name from recall
# - Retain match if score > threshold on any
#Finally, take the specifiers extracted from the recall description and look for an exact match in the following order:
# - Model name or number
# - Serial number
# - UPC
# - Specifiers extracted from the product description

# TODO badly need to optimize; convert to map() instead of nested fors; collapse brands together,
# or try pd.df.apply() to speed this up somehow
# https://github.com/nalepae/pandarallel

def find_match(report, recalls, threshold=60):
    match_ids = []
    for c in list(set(report['candidate_brand'])):
        if pd.isnull(c) or not c:
            continue
        else:
            match_ids += [recalls.iloc[r]['RecallID'] for r in range(len(recalls)) if
                        (pd.Series(list(zip(*process.extract(c, 
                                                             recalls.iloc[r]['candidate_brand'], 
                                                             limit=len(recalls.iloc[r]['candidate_brand']),
                                                            scorer = fuzz.token_set_ratio)))[1]) > threshold).any()]
    if not match_ids:
        return 0
    else:
        candidate_recalls = recalls[recalls['RecallID'].apply(lambda x: x in match_ids)]
        product_match = process.extract(report['Product Type'], list(candidate_recalls['Products_Name']), 
                                        limit=len(candidate_recalls), scorer = fuzz.token_set_ratio)
        v = pd.Series(list(zip(*product_match))[1]).values
        candidate_recalls = candidate_recalls[pd.Series(list(zip(*product_match))[1]).values > threshold]
        if candidate_recalls.empty:
            return 0
        else:
            reported_specs = [report[spec].lower() 
                              if not pd.isnull(report[spec]) else '' 
                              for spec in ['Model Name or Number', 'Serial Number', 'UPC']]
                        
            reported_spec_matches = candidate_recalls['specifiers'].apply(
                lambda x: any([spec.lower() in report_spec.lower() for spec in x for report_spec in reported_specs]))
            
            definite_match = candidate_recalls['RecallID'][reported_spec_matches]
                        
            if len(definite_match) > 1:
                warnings.warn('More than one "unique" match found')
                return definite_match.min()
            if len(definite_match) > 0:
                return definite_match
            else:
                bag_specifier_matches = candidate_recalls['specifiers'].apply(
                lambda x: any([spec.lower() in report_spec.lower() 
                               for spec in x 
                               for report_spec in report['specifiers']]))
                bag_matches = candidate_recalls['RecallID'][bag_specifier_matches]
                if len(bag_matches) > 1:
                    warnings.warn('More than one "unique" match found')
                    return bag_matches.min()
                return bag_matches if bag_specifier_matches.sum() else 0


In [419]:
r = reports.iloc[6, :]
find_match(r, recalls, threshold=80)
#reports.iloc[8:9, :]

344    8381
428    8298
Name: RecallID, dtype: Int64

In [192]:
list(set(reports['candidate_brand'].iloc[200]))

['Bosch', 'BSH HOME APPLIANCES CORPORATION']

In [207]:
list(set(recalls['candidate_brand'].iloc[0]))

[nan, 'Vornado Air Reannounces ', 'Vornado Air LLC, of Andover, Kan.']

In [234]:
recalls.head()

,RecallID,RecallNumber,RecallDate,Description,URL,Title,ConsumerContact,LastPublishDate,Images,SoldAtLabel,Distributors_CompanyID,Distributors_Name,Hazards_HazardType,Hazards_HazardTypeID,Hazards_Name,Importers_CompanyID,Importers_Name,Inconjunctions_URL,Injuries_Name,ManufacturerCountries_Country,Manufacturers_CompanyID,Manufacturers_Name,UPC,Products_CategoryID,Products_Description,Products_Model,Products_Name,Products_NumberOfUnits,Products_Type,Remedies_Name,RemedyOptions_Option,Retailers_CompanyID,Retailers_Name,CompanyShortname,ProductsShortname,HazardAlt,3,candidate_brand,specifiers
344,8381,18199,2018-08-22,This recall involves Vornado VH101 Personal Vo...,https://www.cpsc.gov/Recalls/2018/Vornado-Air-...,Vornado Air Reannounces Recall of Electric Spa...,Vornado toll-free at 855-215-5131 from 8 a.m. ...,2018-08-22,[{'URL': 'https://www.cpsc.gov/s3fs-public/pic...,NaN,NaN,NaN,NaN,NaN,The electric space heater can overheat when in...,NaN,"Vornado Air LLC, of Andover, Kan.",NaN,"In December 2017, a 90-year-old man in Chanhas...",China,NaN,NaN,NaN,NaN,NaN,NaN,VH101 Personal Vortex electric space heaters,350000.0,NaN,Consumers should immediately stop using the re...,Refund,NaN,"Bed Bath & Beyond, Home Depot, Menards, Orchar...",Vornado Air Reannounces,of Electric Space Heaters Following Report of...,None,None,"(Vornado Air Reannounces , nan, nan, Vornado A...","[VH101, 7.8, ETL, 7.10, 7.2]"
428,8298,18136,2018-04-04,This recall involves Vornado VH101 Personal Vo...,https://www.cpsc.gov/Recalls/2018/Vornado-Air-...,Vornado Air Recalls Electric Space Heaters Due...,Vornado toll-free at 855-215-5131 from 8 a.m. ...,2018-04-04,[{'URL': 'https://www.cpsc.gov/s3fs-public/Pic...,NaN,NaN,NaN,NaN,NaN,The electric space heater can overheat when in...,NaN,"Vornado Air LLC, of Andover, Kan.",NaN,Vornado has received 15 reports of the heaters...,China,NaN,NaN,NaN,NaN,NaN,NaN,VH101 Personal Vortex electric space heaters,350000.0,NaN,Consumers should immediately stop using the re...,Refund,NaN,"Bed Bath & Beyond, Home Depot, Menards, Orchar...",Vornado Air,Electric Space Heaters,Fire and Burn Hazards,None,"(Vornado Air , nan, nan, Vornado Air LLC, of A...","[VH101, 7.8, ETL, 7.10, 7.2]"
486,8240,18088,2018-01-31,This recall involves Vornado Sunny CS (cribsid...,https://www.cpsc.gov/Recalls/2018/Vornado-Air-...,Vornado Air Recalls Cribside Space Heaters Due...,Contact Vornado toll-free at 844-202-7978 from...,2018-01-31,[{'URL': 'https://www.cpsc.gov/s3fs-public/fan...,NaN,NaN,"Vornado Air LLC, of Andover, Kan.",NaN,NaN,A broken motor mount can allow the electric he...,NaN,"Vornado Air LLC, of Andover, Kan.",http://healthycanadians.gc.ca/recall-alert-rap...,Vornado has received five reports of the elect...,China,NaN,NaN,NaN,NaN,NaN,NaN,Sunny CS (cribside) nursery space heaters,5000.0,NaN,Consumers should immediately stop using the re...,Replace,NaN,"Bed Bath & Beyond, buybuy Baby and other store...",Vornado Air,Cribside Space Heaters,Fire and Burn Hazards,None,"(Vornado Air , nan, Vornado Air LLC, of Andove...","[12, EH1-0090, XXX17-XXXXXX, 11, CS, SUNNY]"
1518,1748,14256,2014-08-14,This recall involves Vornado VH110 Whole Room ...,https://www.cpsc.gov/Recalls/2014/Vornado-Air-...,Vornado Air Recalls Electric Space Heaters Due...,Vornado toll-free at (844) 205-7978 from 8 a.m...,2015-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/Rec...,NaN,0,"Vornado Air, LLC, of Andover, Kan.",NaN,47,The heater can overheat and cause the units to...,0,"Vornado Air, LLC, of Andover, Kan.",NaN,Vornado has received 29 reports of units overh...,China,NaN,Vornado Air,NaN,1924,NaN,NaN,Electric space heaters,79000.0,Electric Heaters,Customers should immediately turn off and unpl...,Replace,NaN,"Bed Bath and Beyond, Home Depot, Menards, Orch...",Vornado Air,Electric Space Heaters,Fire and Burn Hazards,None,"(Vornado Air , Vornado Air, Vornado Air, LLC, ...","[VH110, 10.5]"
4992,2662,04191,2004-08-03,"August 3, 2004Release #04-191 Firm's Recall Ho...",https://w

In [147]:
reports = pd.read_csv('SPDB/IncidentReports.csv', encoding="ISO-8859-1", dtype='object')

In [142]:
def get_matched_brands(brand, reports, recalls):
    brand_reports = reports[reports['Manufacturer / Importer / Private Labeler Name'].str.contains(brand, case=False) 
                            | reports['Brand'].str.contains(brand, case=False)
                            | reports['Incident Description'].str.contains(brand, case=False)]
    brand_recalls = recalls[recalls['CompanyShortname'].str.contains(brand, case=False) 
                            | recalls['Manufacturers_Name'].str.contains(brand, case=False) 
                            | recalls['Importers_Name'].str.contains(brand, case=False) 
                            | recalls['Distributors_Name'].str.contains(brand, case=False) 
                            | recalls['Retailers_Name'].str.contains(brand, case=False)]
    return (brand_reports, brand_recalls)


In [151]:
v_reports, v_recalls = get_matched_brands('Vornado', reports, recalls)

In [152]:
b_reports, b_recalls = get_matched_brands('Bosch', reports, recalls)

In [273]:
b_recalls['Products_Name']

4                 Buderus GB125-35 oil-condensing boilers
696                       Roof-mounted Bosch solar panels
999                            Bosch small angle grinders
1212                         Radion wireless smoke alarms
1231                Dremel® MICRO™ Model 8050 Rotary Tool
1697    Bosch Security Systems GV4 Fire Alarm Control ...
2150                      Robert Bosch SkilSaw Miter Saws
2268                             Fire Alarm Control Panel
2656                                  Bosch hammer drills
2755    Bosch Thermotechnology Service kits for gas-fi...
3090    Bosch Thermotechnology Buderus gas-fired, floo...
3169    Bosch Thermotechnology Buderus Gas-Fired Wall-...
3358    BSH Home Appliances Bosch and Siemens Model Di...
3588                      Robert Bosch Tool Hammer Drills
3989                                   Skil Circular Saws
4397    Robert Bosch Tool Vermont American Abrasive Cu...
4398    Robert Bosch Tool Vermont American Masonry Dri...
4729          

In [154]:
recalls

,RecallID,RecallNumber,RecallDate,Description,URL,Title,ConsumerContact,LastPublishDate,Images,SoldAtLabel,Distributors_CompanyID,Distributors_Name,Hazards_HazardType,Hazards_HazardTypeID,Hazards_Name,Importers_CompanyID,Importers_Name,Inconjunctions_URL,Injuries_Name,ManufacturerCountries_Country,Manufacturers_CompanyID,Manufacturers_Name,UPC,Products_CategoryID,Products_Description,Products_Model,Products_Name,Products_NumberOfUnits,Products_Type,Remedies_Name,RemedyOptions_Option,Retailers_CompanyID,Retailers_Name,CompanyShortname,ProductsShortname,HazardAlt,3
0,8726,20056,2020-01-14,This recall involves Thompson’s WaterSeal wate...,https://www.cpsc.gov/Recalls/2020/The-Thompson...,The Thompson’s Company Recalls Aerosol Waterpr...,The Thompson’s Company toll-free at 888-304-37...,2020-01-14,[{'URL': 'https://www.cpsc.gov/s3fs-public/Cap...,NaN,NaN,NaN,NaN,NaN,The contents of the cans can react with the pa...,NaN,NaN,NaN,The Thompson’s Company received approximately ...,United States,NaN,"The Thompson’s Company, of Cleveland, Ohio",NaN,NaN,NaN,NaN,Thompson’s® WaterSeal® Waterproofing Wood Prot...,852000.0,NaN,Consumers should immediately stop using Thomps...,Refund,NaN,"Lowe’s Home Improvement, Walmart, Menards, Hom...",The Thompson’s Company,Aerosol Waterproofing Wood and Masonry Protec...,Fire Hazard,None
1,8723,20050,2020-01-09,This recall involves Holiday Travel mugs with ...,https://www.cpsc.gov/Recalls/2020/Boston-Wareh...,Boston Warehouse Trading Corp. Recalls Holiday...,Boston Warehouse Trading Corp. toll-free at 88...,2020-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,The mugs are mislabeled as microwave safe. If ...,NaN,"Meijer Distribution Inc., of Grand Rapids, Mich.",NaN,The firm has received one report of sparks whe...,China,NaN,"Boston Warehouse Trading Corp., of Norwood, Mass.",NaN,NaN,NaN,NaN,Holiday Travel Mugs,2400.0,NaN,Consumers should immediately stop using the re...,Refund,NaN,Exclusively at Meijer stores nationwide from O...,Boston Warehouse Trading Corp.,Holiday Travel Mugs,Fire Hazard; Sold Exclusively at Meijer Stores,None
2,8724,20051,2020-01-09,This recall involves the Libbey Glass 33.5 oz....,https://www.cpsc.gov/Recalls/2020/Libbey-Glass...,Libbey Glass Recalls Milk Bottles Due to Lacer...,Libbey Glass at 800-982-7063 between 8 a.m. an...,2020-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,"The bottles can break unexpectedly during use,...",NaN,"Libbey Glass Inc., of Toledo, Ohio",NaN,None Reported,China,NaN,NaN,NaN,NaN,NaN,NaN,33.5 oz. Milk Bottles,44300.0,NaN,Foodservice establishments and customers shoul...,Refund,NaN,Libby sold the recalled bottles to various foo...,Libbey Glass,Milk Bottles,Laceration Hazard,None
3,8725,20712,2020-01-09,This recall involves Little Bambino 4 in 1 can...,https://www.cpsc.gov/Recalls/2020/Thesaurus-Gl...,Thesaurus Global Marketing Recalls Tricycles D...,Little Bambino toll-free at 866-633-8202 from ...,2020-01-09,[{'URL': 'https://www.cpsc.gov/s3fs-public/1_4...,NaN,NaN,NaN,NaN,NaN,Paint on the canopy’s frame contains levels of...,NaN,"Thesaurus Global Marketing Inc., of Doral, FL",NaN,None reported,China,NaN,NaN,NaN,NaN,NaN,NaN,Little Bambino 4 in 1 canopy children’s tricycles,370.0,NaN,Consumers should immediately stop using the re...,Refund,NaN,Amazon.com from October 2018 through June 2019...,Thesaurus Global Marketing,Tricycles,Violation of the Federal Lead Paint Ban; Risk...,Alert)
4,8721,20046,2019-12-20,This recall involves Buderus brand GB125-35 oi...,https://www.cpsc.gov/Recalls/2020/Bosch-Thermo...,Bosch Thermotechnology Recalls Buderus Boilers...,Bosch Thermotechnology at 800-323-1943 from 8 ...,2019-12-20,[{'URL': 'https://www.cpsc.gov/s3fs-public/Scr...,NaN,NaN,"Bosch Thermotechnology Corp., of Watertown, Mass.",NaN,NaN,"The siphon can become blocked, leading to a de...",NaN,"Bosch Thermotechnology Corp., of Watertown, Mass.",NaN,None reported in the U.S.,Germany,NaN,"Bosch Thermotechnik